In [2]:
# !pip install datasets

In [3]:
from datasets import load_dataset

dataset = load_dataset("noahcao/dancetrack")


In [ ]:
!pip install -r requirement.txt

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --timeout 1000

In [ ]:
!git clone https://github.com/samson-wang/cython_bbox.git
%cd cython_bbox
!pip install .


In [ ]:
python demo.py --video_path WednesdayDance.mp4 --output_video_path outputs/siren/img1 --weight DanceTrack_yolox_s.tar --yolox_type yolox-s --output_txt_path outputs --config ./configs/demo_test.yaml --dataset DanceTrack

In [22]:
cd D:\Project\diffusiontrack_MOT\DiffMOT_Choi

D:\Project\diffusiontrack_MOT\DiffMOT_Choi


In [23]:
import os.path as osp
import os
import numpy as np
import requests
import glob as gb
import argparse
import cv2
from PIL import Image
import subprocess
from diffmot import DiffMOT
import argparse
import yaml
from easydict import EasyDict

In [24]:
def extract_frames(video_path, output_folder, target_width=640):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    if video_path.endswith(".gif"):
        with Image.open(video_path) as img:
            while True:
                try:
                    frame_count = img.tell()
                    frame_img = img.convert("RGB")
                    frame_img.save(os.path.join(output_folder, f"{frame_count+1:05d}.jpg"))
                    img.seek(frame_count + 1)
                except EOFError:
                    break
    else:
        # Open the video file
        cap = cv2.VideoCapture(video_path)

        # Get original video properties
        original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Calculate the scale factor
        scale_factor = target_width / original_width
        target_height = int(original_height * scale_factor)

        # Variable to keep track of frame count
        frame_count = 0

        # Read frames until there are no more
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Save the frame as an image
            frame_path = os.path.join(output_folder, f"{frame_count+1:05d}.jpg")

            # Resize the frame
            resized_frame = cv2.resize(frame, (target_width, target_height))
            cv2.imwrite(frame_path, resized_frame)

            frame_count += 1

        # Release the video capture object
        cap.release()

In [25]:
def mkdirs(d):
    if not osp.exists(d):
        os.makedirs(d)

def download_weights(weight_root="weights", file_name="", url=""):
    mkdirs(weight_root)

    # Specify the local file path where you want to save the downloaded file
    local_file_path = os.path.join(weight_root, file_name)

    # Check if the file already exists
    if os.path.exists(local_file_path):
        print(f"> File '{local_file_path}' already exists. Skipping download.")
        return

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the content of the response (file data)
        file_content = response.content

        # Specify the local file path where you want to save the downloaded file
        local_file_path = os.path.join(weight_root, file_name)

        # Write the file content to the local file
        with open(local_file_path, 'wb') as f:
            f.write(file_content)

        print(f"> File downloaded successfully to '{local_file_path}'")
    else:
        print(f"> Failed to download file: {response.status_code} - {response.reason}")

In [26]:
# Create a simple class to hold arguments
class Args:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

# Initialize args with your parameters
args = Args(
    video_path="WednesdayDance.mp4",  # replace with actual video path
    output_video_path="outputs/siren/img1",  # replace with actual output path
    output_txt_path="outputs",  # replace with actual txt path
    weight="DanceTrack_yolox_s.tar",  # specify the YOLO weight you want
    yolox_type="yolox-s",  # specify the YOLOX type as needed
    config="./configs/demo_test.yaml",  # replace with actual config path
    dataset="DanceTrack"  # replace with your dataset name
)

In [6]:
def main(args):
    with open(args.config) as f:
       config = yaml.safe_load(f)

    for k, v in vars(args).items():
       config[k] = v
    config["exp_name"] = args.config.split("/")[-1].split(".")[0]
    config["dataset"] = args.dataset
    config = EasyDict(config)
    agent = DiffMOT(config)

    agent.eval()

    # if config["eval_mode"]:
    #     agent.eval()
    # else:
    #     agent.train()

In [7]:
extract_frames(video_path=args.video_path, output_folder=args.output_video_path)

In [7]:
# Download re_id
download_weights(weight_root="external/weights", file_name="dance_sbs_S50.pth", url="https://github.com/Kroery/DiffMOT/releases/download/v1.0/dance_sbs_S50.pth")
download_weights(weight_root="external/weights", file_name="mot20_sbs_S50.pth", url="https://github.com/Kroery/DiffMOT/releases/download/v1.0/mot20_sbs_S50.pth")


> File 'external/weights\dance_sbs_S50.pth' already exists. Skipping download.
> File 'external/weights\mot20_sbs_S50.pth' already exists. Skipping download.


In [8]:
# Download DiffMOT
DiffMOT_prefix = "MOT" if "MOT" in args.dataset else args.dataset
download_weights(weight_root="experiments/mot_ddm_1000_deeper", file_name=f"{args.dataset}_epoch800.pt", url=f"https://github.com/Kroery/DiffMOT/releases/download/v1.0/{DiffMOT_prefix}_epoch800.pt")


> File 'experiments/mot_ddm_1000_deeper\DanceTrack_epoch800.pt' already exists. Skipping download.


In [9]:
yolo_weight_map = {
    "DanceTrack_yolox_s.tar" : "https://github.com/Kroery/DiffMOT/releases/download/v1.0/DanceTrack_yolox_s.tar",
    "DanceTrack_yolox_m.tar" : "https://github.com/Kroery/DiffMOT/releases/download/v1.0/DanceTrack_yolox_m.tar",
    "DanceTrack_yolox_l.tar" : "https://github.com/Kroery/DiffMOT/releases/download/v1.0/DanceTrack_yolox_l.tar",
    "DanceTrack_yolox_x.tar" : "https://github.com/Kroery/DiffMOT/releases/download/v1.0/DanceTrack_yolox_x.tar",
    "MOT17_yolox_x.tar" : "https://github.com/Kroery/DiffMOT/releases/download/v1.0/MOT17_yolox_x.tar",
    "MOT20_yolox_x.tar" : "https://github.com/Kroery/DiffMOT/releases/download/v1.0/MOT20_yolox_x.tar",
    "SportsMOT_yolox_x.tar" : "https://github.com/Kroery/DiffMOT/releases/download/v1.0/SportsMOT_yolox_x.tar",
    "SportsMOT_yolox_x_mix.tar" : "https://github.com/Kroery/DiffMOT/releases/download/v1.0/SportsMOT_yolox_x_mix.tar"
}    

# Download YOLOX
download_weights(file_name=args.weight, url=yolo_weight_map[args.weight])

> File 'weights\DanceTrack_yolox_s.tar' already exists. Skipping download.


In [10]:
# Define the command to run the Python script
command = ["python", "external/YOLOX/tools/demo.py", "image", 
            "-n", args.yolox_type, 
            "-c", f"weights/{args.weight}", 
            "--path", args.output_video_path, 
            "--output_txt_path", args.output_txt_path,
            "--conf", "0.45", "--nms", "0.45", "--tsize", "640", "--num_classes", "1", 
            "--legacy", "--save_result", "--save_txt", "--device", "gpu"]

# Run the command
result = subprocess.run(command, capture_output=True, text=True)

In [11]:
# !pip install git+https://github.com/Megvii-BaseDetection/YOLOX.git


In [12]:
# Check if the command was successful
if result.returncode == 0:
    print("Python script executed successfully!")
else:
    print("Error running Python script:")
    print(result.stderr)



Python script executed successfully!


In [14]:
# main(args=args)

In [12]:
with open(args.config) as f:
    config = yaml.safe_load(f)

config

{'eps': 0.001,
 'eval_mode': True,
 'lr': 0.0001,
 'data_dir': './outputs',
 'diffnet': 'HMINet',
 'interval': 5,
 'augment': True,
 'encoder_dim': 256,
 'tf_layer': 3,
 'epochs': 800,
 'batch_size': 2048,
 'seed': 123,
 'eval_every': 20,
 'gpus': [0, 1, 2, 3],
 'eval_at': 800,
 'det_dir': './outputs/detections/',
 'info_dir': './outputs',
 'reid_dir': 'weights/embedding',
 'save_dir': 'outputs/yolox_m_lt_ddm_1000eps_deeper_800_1rev',
 'eval_expname': 'mot_ddm_1000_deeper',
 'high_thres': 0.4,
 'low_thres': 0.1,
 'w_assoc_emb': 2.2,
 'aw_param': 1.7,
 'preprocess_workers': 16,
 'device': 'cuda',
 'eval_device': 'None'}

In [16]:
config.diffnet

'HMINet'

In [15]:
with open(args.config) as f:
    config = yaml.safe_load(f)

for k, v in vars(args).items():
    config[k] = v
config["exp_name"] = args.config.split("/")[-1].split(".")[0]
config["dataset"] = args.dataset
config = EasyDict(config)

In [24]:
import logging

# Configure the logger
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [ ]:
from collections import OrderedDict
from pathlib import Path
import os
import pickle

import torch
import cv2
import torchvision
import torchreid
import numpy as np

from external.adaptors.fastreid_adaptor import FastReID


class EmbeddingComputer:
    def __init__(self, config, dataset, test_dataset, grid_off, max_batch=1024):
        self.model = None
        self.dataset = dataset
        self.test_dataset = test_dataset
        self.crop_size = (128, 384)
        os.makedirs(config.reid_dir, exist_ok=True)
        self.cache_path = os.path.join(config.reid_dir, "{}_embedding.pkl")


        self.cache = {}
        self.cache_name = ""
        self.grid_off = grid_off
        self.max_batch = max_batch

        # Only used for the general ReID model (not FastReID)
        self.normalize = False

    def load_cache(self, path):
        self.cache_name = path
        cache_path = self.cache_path.format(path)
        if os.path.exists(cache_path):
            with open(cache_path, "rb") as fp:
                self.cache = pickle.load(fp)

    def get_horizontal_split_patches(self, image, bbox, tag, idx, viz=False):
        if isinstance(image, np.ndarray):
            h, w = image.shape[:2]
        else:
            h, w = image.shape[2:]

        bbox = np.array(bbox)
        bbox = bbox.astype(np.int)
        if bbox[0] < 0 or bbox[1] < 0 or bbox[2] > w or bbox[3] > h:
            # Faulty Patch Correction
            bbox[0] = np.clip(bbox[0], 0, None)
            bbox[1] = np.clip(bbox[1], 0, None)
            bbox[2] = np.clip(bbox[2], 0, image.shape[1])
            bbox[3] = np.clip(bbox[3], 0, image.shape[0])

        x1, y1, x2, y2 = bbox
        w = x2 - x1
        h = y2 - y1
        ### TODO - Write a generalized split logic
        split_boxes = [
            [x1, y1, x1 + w, y1 + h / 3],
            [x1, y1 + h / 3, x1 + w, y1 + (2 / 3) * h],
            [x1, y1 + (2 / 3) * h, x1 + w, y1 + h],
        ]

        split_boxes = np.array(split_boxes, dtype="int")
        patches = []
        # breakpoint()
        for ix, patch_coords in enumerate(split_boxes):
            if isinstance(image, np.ndarray):
                im1 = image[patch_coords[1] : patch_coords[3], patch_coords[0] : patch_coords[2], :]

                if viz:  ## TODO - change it from torch tensor to numpy array
                    dirs = "./viz/{}/{}".format(tag.split(":")[0], tag.split(":")[1])
                    Path(dirs).mkdir(parents=True, exist_ok=True)
                    cv2.imwrite(
                        os.path.join(dirs, "{}_{}.png".format(idx, ix)),
                        im1.squeeze(0).permute(1, 2, 0).detach().cpu().numpy() * 255,
                    )
                patch = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
                patch = cv2.resize(patch, self.crop_size, interpolation=cv2.INTER_LINEAR)
                patch = torch.as_tensor(patch.astype("float32").transpose(2, 0, 1))
                patch = patch.unsqueeze(0)
                # print("test ", patch.shape)
                patches.append(patch)
            else:
                im1 = image[:, :, patch_coords[1] : patch_coords[3], patch_coords[0] : patch_coords[2]]
                patch = torchvision.transforms.functional.resize(im1, (256, 128))
                patches.append(patch)

        patches = torch.cat(patches, dim=0)

        # print("Patches shape ", patches.shape)
        # patches = np.array(patches)
        # print("ALL SPLIT PATCHES SHAPE - ", patches.shape)

        return patches

    def compute_embedding(self, img, bbox, tag):
        if self.cache_name != tag.split(":")[0]:
            self.load_cache(tag.split(":")[0])

        # if tag in self.cache:
        #     embs = self.cache[tag]
        #     if embs.shape[0] != bbox.shape[0]:
        #         self.cache.clear()  # Clear the cache dictionary
        #         raise RuntimeError(
        #             f"ERROR: The number of cached embeddings : {embs.shape[0]} don't match the "
        #             f"number of detections : {bbox.shape[0]}.\nWas the detector model changed? Delete cache if so."
        #         )
        #     return embs
        
        if tag in self.cache:
            embs = self.cache[tag]
            if embs.shape[0] != bbox.shape[0]:
                logger.warning(f"Cached embeddings: {embs.shape[0]}, Detections: {bbox.shape[0]} for tag: {tag}. Clearing cache.")
                self.cache.clear()  # Clear the cache dictionary
                return self.compute_embedding(img, bbox, tag)  # Recompute embeddings after clearing cache
            return embs

        if self.model is None:
            self.initialize_model()

        # Generate all of the patches
        crops = []
        if self.grid_off:
            # Basic embeddings
            h, w = img.shape[:2]
            results = np.round(bbox).astype(np.int32)
            # print(results)

            results[:, 0] = results[:, 0].clip(0, w)
            results[:, 1] = results[:, 1].clip(0, h)
            results[:, 2] = results[:, 2].clip(0, w)
            results[:, 3] = results[:, 3].clip(0, h)

            crops = []
            img_height, img_width = img.shape[:2]
            for p in results:

                x1, y1, x2, y2 = p[0], p[1], p[2], p[3]
    
                # Ensure coordinates are within image bounds
                if x1 >= x2 or y1 >= y2 or x1 < 0 or y1 < 0 or x2 > img_width or y2 > img_height:
                    print(f"Skipping invalid crop region: {p}")
                    continue;

                # crop = img[p[1] : p[3], p[0] : p[2]]
                crop = img[y1:y2, x1:x2]
                # print(p)
                crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                crop = cv2.resize(crop, self.crop_size, interpolation=cv2.INTER_LINEAR).astype(np.float32)
                if self.normalize:
                    crop /= 255
                    crop -= np.array((0.485, 0.456, 0.406))
                    crop /= np.array((0.229, 0.224, 0.225))
                crop = torch.as_tensor(crop.transpose(2, 0, 1))
                crop = crop.unsqueeze(0)
                crops.append(crop)
        else:
            # Grid patch embeddings
            for idx, box in enumerate(bbox):
                crop = self.get_horizontal_split_patches(img, box, tag, idx)
                crops.append(crop)

        if not crops:
            print(f"No valid crops were generated for tag {tag}. Returning empty embeddings.")
            return np.empty((0, 265))  # Adjust dimensions as necessary

        crops = torch.cat(crops, dim=0)

        # Create embeddings and l2 normalize them
        embs = []
        for idx in range(0, len(crops), self.max_batch):
            batch_crops = crops[idx : idx + self.max_batch]
            batch_crops = batch_crops.cuda()
            with torch.no_grad():
                batch_embs = self.model(batch_crops)
            embs.extend(batch_embs)
        embs = torch.stack(embs)
        embs = torch.nn.functional.normalize(embs, dim=-1)

        if not self.grid_off:
            embs = embs.reshape(bbox.shape[0], -1, embs.shape[-1])
        embs = embs.cpu().numpy()

        self.cache[tag] = embs
        return embs

    def initialize_model(self):
        if self.dataset == "mot17":
            if self.test_dataset:
                path = "external/weights/mot17_sbs_S50.pth"
            else:
                return self._get_general_model()
        elif self.dataset == "mot20":
            if self.test_dataset:
                path = "external/weights/mot20_sbs_S50.pth"
            else:
                return self._get_general_model()
        elif self.dataset == "dancetrack":
            path = "external/weights/dance_sbs_S50.pth"
            # path = "/home/estar/lwy/DiffMOT/external/weights/dancetrack_sbs_S50_hybtid.pth"
        elif self.dataset == "sports":
            path = "/home/estar/lwy/BoT-SORT-main/fast_reid/tools/logs/SportsMOT/sbs_S50/model_0058.pth"
        else:
            raise RuntimeError("Need the path for a new ReID model.")

        model = FastReID(path)
        model.eval()
        model.cuda()
        model.half()
        self.model = model

    def _get_general_model(self):
        """Used for the half-val for MOT17/20.

        The MOT17/20 SBS models are trained over the half-val we
        evaluate on as well. Instead we use a different model for
        validation.
        """
        model = torchreid.models.build_model(name="osnet_ain_x1_0", num_classes=2510, loss="softmax", pretrained=False)
        sd = torch.load("external/weights/osnet_ain_ms_d_c.pth.tar")["state_dict"]
        new_state_dict = OrderedDict()
        for k, v in sd.items():
            name = k[7:]  # remove `module.`
            new_state_dict[name] = v
        # load params
        model.load_state_dict(new_state_dict)
        model.eval()
        model.cuda()
        self.model = model
        self.crop_size = (128, 256)
        self.normalize = True

    def dump_cache(self):
        if self.cache_name:
            with open(self.cache_path.format(self.cache_name), "wb") as fp:
                pickle.dump(self.cache, fp)


In [31]:
import itertools
import os
import os.path as osp
import time
from collections import deque

import cv2
import numpy as np
import torch
import torch.nn.functional as F
from models import *

from tracking_utils.kalman_filter import KalmanFilter
from tracking_utils.log import logger
from tracking_utils.utils import *

from tracker import matching

from tracker.basetrack import BaseTrack, TrackState

from tracker.cmc import CMCComputer
from tracker.gmc import GMC
# from .embedding import EmbeddingComputer


class STrack(BaseTrack):
    # shared_kalman = KalmanFilter()
    def __init__(self, tlwh, score, temp_feat=None, buffer_size=30):

        # wait activate
        self.xywh_omemory = deque([], maxlen=buffer_size)
        self.xywh_pmemory = deque([], maxlen=buffer_size)
        self.xywh_amemory = deque([], maxlen=buffer_size)

        self.conds = deque([], maxlen=5)


        self._tlwh = np.asarray(tlwh, dtype=float)
        self.kalman_filter = None
        self.mean, self.covariance = None, None
        self.is_activated = False

        self.score = score
        self.tracklet_len = 0

        self.emb = temp_feat
        self.features = deque([], maxlen=buffer_size)

    def update_features(self, feat, alpha=0.95):
        self.curr_feat = feat
        self.emb = alpha * self.emb + (1 - alpha) * feat
        self.emb /= np.linalg.norm(self.emb)

    def predict(self):
        mean_state = self.mean.copy()
        if self.state != TrackState.Tracked:
            mean_state[7] = 0
        self.mean, self.covariance = self.kalman_filter.predict(mean_state, self.covariance)

    @staticmethod
    def multi_predict(stracks):
        if len(stracks) > 0:
            multi_mean = np.asarray([st.mean.copy() for st in stracks])
            multi_covariance = np.asarray([st.covariance for st in stracks])
            for i, st in enumerate(stracks):
                if st.state != TrackState.Tracked:
                    multi_mean[i][7] = 0
            multi_mean, multi_covariance = STrack.shared_kalman.multi_predict(multi_mean, multi_covariance)
            for i, (mean, cov) in enumerate(zip(multi_mean, multi_covariance)):
                stracks[i].mean = mean
                stracks[i].covariance = cov

    @staticmethod
    def multi_predict_diff(stracks, model, img_w, img_h):
        if len(stracks) > 0:
            dets = np.asarray([st.xywh.copy() for st in stracks]).reshape(-1, 4)

            dets[:, 0::2] = dets[:, 0::2] / img_w
            dets[:, 1::2] = dets[:, 1::2] / img_h

            conds = [st.conds for st in stracks]

            multi_track_pred = model.generate(conds, sample=1, bestof=True, img_w=img_w, img_h=img_h)
            track_pred = multi_track_pred.mean(0)


            track_pred = track_pred + dets

            track_pred[:, 0::2] = track_pred[:, 0::2] * img_w
            track_pred[:, 1::2] = track_pred[:, 1::2] * img_h
            track_pred[:, 0] = track_pred[:, 0] - track_pred[:, 2] / 2
            track_pred[:, 1] = track_pred[:, 1] - track_pred[:, 3] / 2


            for i, st in enumerate(stracks):
                st._tlwh = track_pred[i]
                st.xywh_pmemory.append(st.xywh.copy())
                st.xywh_amemory.append(st.xywh.copy())

                tmp_delta_bbox = st.xywh.copy() - st.xywh_amemory[-2].copy()
                tmp_conds = np.concatenate((st.xywh.copy(), tmp_delta_bbox))
                st.conds.append(tmp_conds)


    def activate(self, frame_id):
        """Start a new tracklet"""
        self.track_id = self.next_id()

        self.tracklet_len = 0
        self.state = TrackState.Tracked
        if frame_id == 1:
            self.is_activated = True

        self.frame_id = frame_id
        self.start_frame = frame_id
        self.xywh_omemory.append(self.xywh.copy())
        self.xywh_pmemory.append(self.xywh.copy())
        self.xywh_amemory.append(self.xywh.copy())


        delta_bbox = self.xywh.copy() - self.xywh.copy()
        tmp_conds = np.concatenate((self.xywh.copy(), delta_bbox))
        self.conds.append(tmp_conds)

    def re_activate(self, new_track, frame_id, new_id=False):
        new_tlwh = new_track.tlwh
        self._tlwh = new_tlwh
        self.xywh_omemory.append(self.xywh.copy())
        self.xywh_amemory[-1] = self.xywh.copy()

        tmp_delta_bbox = self.xywh.copy() - self.xywh_amemory[-2].copy()
        tmp_conds = np.concatenate((self.xywh.copy(), tmp_delta_bbox))
        self.conds[-1] = tmp_conds

        self.tracklet_len = 0
        self.state = TrackState.Tracked
        self.is_activated = True
        self.frame_id = frame_id
        if new_id:
            self.track_id = self.next_id()

    def update(self, new_track, frame_id, update_feature=False):
        """
        Update a matched track
        :type new_track: STrack
        :type frame_id: int
        :type update_feature: bool
        :return:
        """
        self.frame_id = frame_id
        self.tracklet_len += 1

        new_tlwh = new_track.tlwh
        self._tlwh = new_tlwh
        self.xywh_omemory.append(self.xywh.copy())
        self.xywh_amemory[-1] = self.xywh.copy()

        if self.is_activated == True:
            tmp_delta_bbox = self.xywh.copy() - self.xywh_amemory[-2].copy()
            tmp_conds = np.concatenate((self.xywh.copy(), tmp_delta_bbox))
            self.conds[-1] = tmp_conds
        else:
            tmp_delta_bbox = self.xywh.copy() - self.xywh_omemory[-2].copy()
            tmp_conds = np.concatenate((self.xywh.copy(), tmp_delta_bbox))
            self.conds[-1] = tmp_conds

        self.state = TrackState.Tracked
        self.is_activated = True

        self.score = new_track.score
        if update_feature:
            self.update_features(new_track.curr_feat)

    @property
    def tlwh(self):
        """Get current position in bounding box format `(top left x, top left y,
                width, height)`.
        """
        if self.mean is None:
            return self._tlwh.copy()
        ret = self.mean[:4].copy()
        ret[2] *= ret[3]
        ret[:2] -= ret[2:] / 2
        return ret

    @property
    def tlbr(self):
        """Convert bounding box to format `(min x, min y, max x, max y)`, i.e.,
        `(top left, bottom right)`.
        """
        ret = self.tlwh.copy()
        ret[2:] += ret[:2]
        return ret

    @property
    def xywh(self):
        """Convert bounding box to format `(min x, min y, max x, max y)`, i.e.,
        `(top left, bottom right)`.
        """
        ret = self.tlwh.copy()
        ret[:2] = ret[:2] + ret[2:] / 2
        # ret[2:] += ret[:2]
        return ret

    @staticmethod
    def tlwh_to_xyah(tlwh):
        """Convert bounding box to format `(center x, center y, aspect ratio,
        height)`, where the aspect ratio is `width / height`.
        """
        ret = np.asarray(tlwh).copy()
        ret[:2] += ret[2:] / 2
        ret[2] /= ret[3]
        return ret

    def to_xyah(self):
        return self.tlwh_to_xyah(self.tlwh)

    @staticmethod
    def tlbr_to_tlwh(tlbr):
        ret = np.asarray(tlbr).copy()
        ret[2:] -= ret[:2]
        return ret

    @staticmethod
    def tlwh_to_tlbr(tlwh):
        ret = np.asarray(tlwh).copy()
        ret[2:] += ret[:2]
        return ret

    def __repr__(self):
        return 'OT_{}_({}-{})'.format(self.track_id, self.start_frame, self.end_frame)


class diffmottracker(object):
    def __init__(self, config, frame_rate=30):
        self.config = config
        self.tracked_stracks = []  # type: list[STrack]
        self.lost_stracks = []  # type: list[STrack]
        self.removed_stracks = []  # type: list[STrack]

        self.frame_id = 0
        self.det_thresh = self.config.high_thres

        self.buffer_size = int(frame_rate / 30.0 * 30)
        self.max_time_lost = self.buffer_size

        self.mean = np.array([0.408, 0.447, 0.470], dtype=np.float32).reshape(1, 1, 3)
        self.std = np.array([0.289, 0.274, 0.278], dtype=np.float32).reshape(1, 1, 3)

        # self.kalman_filter = KalmanFilter()
        self.embedder = EmbeddingComputer(self.config, self.config.dataset.lower(), self.config.eval_mode, True)
        self.alpha_fixed_emb = 0.95


    def dump_cache(self):
        # self.cmc.dump_cache()
        self.embedder.dump_cache()

    def update(self, dets_norm, model, frame_id, img_w, img_h, tag, img=None):
        self.model = model
        self.frame_id += 1
        activated_starcks = []
        refind_stracks = []
        lost_stracks = []
        removed_stracks = []
        dets = dets_norm.copy()
        dets[:, 2] = dets[:, 0] + dets[:, 2]
        dets[:, 3] = dets[:, 1] + dets[:, 3]
        remain_inds = dets[:, 4] > self.det_thresh
        inds_low = dets[:, 4] > self.config.low_thres
        inds_high = dets[:, 4] < self.det_thresh
        inds_second = np.logical_and(inds_low, inds_high)
        dets_second = dets[inds_second]
        dets = dets[remain_inds]


        dets_embs = np.ones((dets.shape[0], 1))
        if dets.shape[0] != 0:
            dets_embs = self.embedder.compute_embedding(img, dets[:, :4], tag)
        trust = (dets[:, 4] - self.det_thresh) / (1 - self.det_thresh)
        af = self.alpha_fixed_emb
        # From [self.alpha_fixed_emb, 1], goes to 1 as detector is less confident
        dets_alpha = af + (1 - af) * (1 - trust)


        if len(dets) > 0:
            '''Detections'''
            detections = [STrack(STrack.tlbr_to_tlwh(tlbrs[:4]), tlbrs[4], f, 30) for
                          (tlbrs, f) in zip(dets[:, :5], dets_embs)]
        else:
            detections = []

        ''' Add newly detected tracklets to tracked_stracks'''
        unconfirmed = []
        tracked_stracks = []  # type: list[STrack]
        for track in self.tracked_stracks:
            if not track.is_activated:
                unconfirmed.append(track)
            else:
                tracked_stracks.append(track)

        ''' Step 2: First association, with embedding'''
        strack_pool = joint_stracks(tracked_stracks, self.lost_stracks)
        STrack.multi_predict_diff(strack_pool, self.model, img_w, img_h)

        trk_embs = [st.emb for st in strack_pool]
        trk_embs = np.array(trk_embs)
        emb_cost = None if (trk_embs.shape[0] == 0 or dets_embs.shape[0] == 0) else trk_embs @ dets_embs.T


        dists = matching.iou_distance(strack_pool, detections)
        iou_matrix = 1 - dists

        if min(iou_matrix.shape) > 0:
            a = (iou_matrix > 0.1).astype(np.int32)
            if a.sum(1).max() == 1 and a.sum(0).max() == 1:
                matched_indices = np.stack(np.where(a), axis=1)
            else:
                if emb_cost is None:
                    emb_cost = 0
                w_assoc_emb = self.config.w_assoc_emb
                aw_param = self.config.aw_param

                w_matrix = matching.compute_aw_new_metric(emb_cost, w_assoc_emb, aw_param)
                emb_cost *= w_matrix

                final_cost = -(iou_matrix + emb_cost)
                matched_indices = matching.linear_assignment2(final_cost)
        else:
            matched_indices = np.empty(shape=(0, 2))


        unmatched_detections = []
        for d, det in enumerate(detections):
            if d not in matched_indices[:, 1]:
                unmatched_detections.append(d)
        unmatched_trackers = []
        for t, trk in enumerate(strack_pool):
            if t not in matched_indices[:, 0]:
                unmatched_trackers.append(t)

        # filter out matched with low IOU
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] < 0.1:
                unmatched_detections.append(m[1])
                unmatched_trackers.append(m[0])
            else:
                matches.append(m.reshape(1, 2))
        if len(matches) == 0:
            matches = np.empty((0, 2), dtype=int)
        else:
            matches = np.concatenate(matches, axis=0)

        u_track = np.array(unmatched_trackers)
        u_detection = np.array(unmatched_detections)


        for itracked, idet in matches:
            track = strack_pool[itracked]
            det = detections[idet]
            alp = dets_alpha[idet]
            if track.state == TrackState.Tracked:
                track.update(det, self.frame_id)
                track.update_features(det.emb, alp)
                activated_starcks.append(track)
            else:
                track.re_activate(det, self.frame_id, new_id=False)
                track.update_features(det.emb, alp)
                refind_stracks.append(track)


        if len(dets_second) > 0:
            '''Detections'''
            detections_second = [STrack(STrack.tlbr_to_tlwh(tlbrs[:4]), tlbrs[4], buffer_size=30) for
                                 (tlbrs) in dets_second[:, :5]]
        else:
            detections_second = []

        r_tracked_stracks = [strack_pool[i] for i in u_track if strack_pool[i].state == TrackState.Tracked]
        dists = matching.iou_distance(r_tracked_stracks, detections_second)
        matches, u_track, u_detection_second = matching.linear_assignment(dists, thresh=0.5)
        for itracked, idet in matches:
            track = r_tracked_stracks[itracked]
            det = detections_second[idet]
            if track.state == TrackState.Tracked:
                track.update(det, self.frame_id)
                activated_starcks.append(track)
            else:
                track.re_activate(det, self.frame_id, new_id=False)
                refind_stracks.append(track)


        for it in u_track:
            # track = strack_pool[it]
            track = r_tracked_stracks[it]
            if not track.state == TrackState.Lost:
                track.mark_lost()
                lost_stracks.append(track)

        '''Deal with unconfirmed tracks, usually tracks with only one beginning frame'''
        detections = [detections[i] for i in u_detection]
        dists = matching.iou_distance(unconfirmed, detections)
        matches, u_unconfirmed, u_detection = matching.linear_assignment(dists, thresh=0.7)
        for itracked, idet in matches:
            alp = dets_alpha[idet]
            unconfirmed[itracked].update(detections[idet], self.frame_id)
            unconfirmed[itracked].update_features(detections[idet].emb, alp)

            activated_starcks.append(unconfirmed[itracked])
        for it in u_unconfirmed:
            track = unconfirmed[it]
            track.mark_removed()
            removed_stracks.append(track)

        """ Step 4: Init new stracks"""
        for inew in u_detection:
            track = detections[inew]
            if track.score < self.det_thresh:
                continue
            # track.activate(self.kalman_filter, self.frame_id)
            track.activate(self.frame_id)
            activated_starcks.append(track)
        """ Step 5: Update state"""
        for track in self.lost_stracks:
            if self.frame_id - track.end_frame > self.max_time_lost:
                track.mark_removed()
                removed_stracks.append(track)

        # print('Ramained match {} s'.format(t4-t3))

        self.tracked_stracks = [t for t in self.tracked_stracks if t.state == TrackState.Tracked]
        self.tracked_stracks = joint_stracks(self.tracked_stracks, activated_starcks)
        self.tracked_stracks = joint_stracks(self.tracked_stracks, refind_stracks)
        self.lost_stracks = sub_stracks(self.lost_stracks, self.tracked_stracks)
        self.lost_stracks.extend(lost_stracks)
        self.lost_stracks = sub_stracks(self.lost_stracks, self.removed_stracks)
        self.removed_stracks.extend(removed_stracks)
        self.tracked_stracks, self.lost_stracks = remove_duplicate_stracks(self.tracked_stracks, self.lost_stracks)
        # get scores of lost tracks
        output_stracks = [track for track in self.tracked_stracks if track.is_activated]


        return output_stracks


def joint_stracks(tlista, tlistb):
    exists = {}
    res = []
    for t in tlista:
        exists[t.track_id] = 1
        res.append(t)
    for t in tlistb:
        tid = t.track_id
        if not exists.get(tid, 0):
            exists[tid] = 1
            res.append(t)
    return res


def sub_stracks(tlista, tlistb):
    stracks = {}
    for t in tlista:
        stracks[t.track_id] = t
    for t in tlistb:
        tid = t.track_id
        if stracks.get(tid, 0):
            del stracks[tid]
    return list(stracks.values())


def remove_duplicate_stracks(stracksa, stracksb):
    pdist = matching.iou_distance(stracksa, stracksb)
    pairs = np.where(pdist < 0.15)
    # pairs = np.where(pdist < 0.)
    dupa, dupb = list(), list()
    for p, q in zip(*pairs):
        timep = stracksa[p].frame_id - stracksa[p].start_frame
        timeq = stracksb[q].frame_id - stracksb[q].start_frame
        if timep > timeq:
            dupb.append(q)
        else:
            dupa.append(p)
    resa = [t for i, t in enumerate(stracksa) if not i in dupa]
    resb = [t for i, t in enumerate(stracksb) if not i in dupb]
    return resa, resb


In [19]:
import os
import torch
import cv2

import numpy as np
import os.path as osp
import logging
from torch import nn, optim, utils
from tensorboardX import SummaryWriter
from tqdm.auto import tqdm

from dataset import DiffMOTDataset
from models.autoencoder import D2MP
from models.condition_embedding import History_motion_embedding

import time
# from tracker.DiffMOTtracker import diffmottracker

from tracking_utils.log import logger
from tracking_utils.timer import Timer
from tracking_utils.visualization import plot_tracking

def write_results(filename, results, data_type='mot'):
    if data_type == 'mot':
        save_format = '{frame},{id},{x1},{y1},{w},{h},1,-1,-1,-1\n'
    elif data_type == 'kitti':
        save_format = '{frame} {id} pedestrian 0 0 -10 {x1} {y1} {x2} {y2} -10 -10 -10 -1000 -1000 -1000 -10\n'
    else:
        raise ValueError(data_type)

    with open(filename, 'w') as f:
        for frame_id, tlwhs, track_ids in results:
            if data_type == 'kitti':
                frame_id -= 1
            for tlwh, track_id in zip(tlwhs, track_ids):
                if track_id < 0:
                    continue
                x1, y1, w, h = tlwh
                x2, y2 = x1 + w, y1 + h
                line = save_format.format(frame=frame_id, id=track_id, x1=x1, y1=y1, x2=x2, y2=y2, w=w, h=h)
                f.write(line)
    logger.info('save results to {}'.format(filename))

def mkdirs(d):
    if not osp.exists(d):
        os.makedirs(d)

class DiffMOT():
    def __init__(self, config):
        self.config = config
        torch.backends.cudnn.benchmark = True
        self._build()

    def train(self):
        for epoch in range(1, self.config.epochs + 1):
            self.train_dataset.augment = self.config.augment
            pbar = tqdm(self.train_data_loader, ncols=80)
            for batch in pbar:
                for k in batch:
                    batch[k] = batch[k].to(device='cuda', non_blocking=True)

                train_loss = self.model(batch)
                train_loss = train_loss.mean()

                self.optimizer.zero_grad()
                pbar.set_description(f"Epoch {epoch},  Loss: {train_loss.item():.6f}")
                train_loss.backward()
                self.optimizer.step()

            if epoch % self.config.eval_every == 0:
                checkpoint = {
                    'ddpm': self.model.state_dict(),
                    'epoch': epoch,
                    'optimizer': self.optimizer.state_dict()
                }
                torch.save(checkpoint, osp.join(self.model_dir, f"{self.config.dataset}_epoch{epoch}.pt"))

    def eval(self):
        det_root = self.config.det_dir
        img_root = det_root.replace('/detections/', '/')

        seqs = [s for s in os.listdir(det_root)]
        seqs.sort()

        for seq in seqs:
            print(seq)
            det_path = osp.join(det_root, seq)
            img_path = osp.join(img_root, seq, 'img1')

            info_path = osp.join(self.config.info_dir, seq, 'seqinfo.ini')
            seq_info = open(info_path).read()
            seq_width = int(seq_info[seq_info.find('imWidth=') + 8:seq_info.find('\nimHeight')])
            seq_height = int(seq_info[seq_info.find('imHeight=') + 9:seq_info.find('\nimExt')])

            tracker = diffmottracker(self.config)
            timer = Timer()
            results = []
            frame_id = 0

            frames = [s for s in os.listdir(det_path)]
            frames.sort()
            imgs = [s for s in os.listdir(img_path)]
            imgs.sort()

            video_path = osp.join(self.config.info_dir, seq, 'output.mp4')
            logger.info(f"video save_path is {video_path}")
            vid_writer = cv2.VideoWriter(
                video_path, cv2.VideoWriter_fourcc(*"mp4v"), 30, (int(seq_width), int(seq_height))
            )

            print(f"Number of detection frames: {len(frames)}")
            print(f"Number of images: {len(imgs)}")


            for i, f in enumerate(frames[:len(imgs)]):
                if frame_id % 10 == 0:
                    logger.info('Processing frame {} ({:.2f} fps)'.format(frame_id, 1. / max(1e-5, timer.average_time)))

                timer.tic()
                
                if 'jpg' in f:
                    f_path = osp.join(det_path, f, f.replace('jpg', 'txt')).replace("\\", "/")
                else:
                    continue;


                dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]

                
                # dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]

                im_path = osp.join(img_path, imgs[i])
                img = cv2.imread(im_path)
                tag = f"{seq}:{frame_id+1}"
                # track
                online_targets = tracker.update(dets, self.model, frame_id, seq_width, seq_height, tag, img)
                # online_targets = tracker.update(dets, self.model, frame_id, seq_width, seq_height, tag)
                online_tlwhs = []
                online_ids = []
                for t in online_targets:
                    tlwh = t.tlwh
                    tid = t.track_id
                    online_tlwhs.append(tlwh)
                    online_ids.append(tid)
                timer.toc()
                # save results
                results.append((frame_id + 1, online_tlwhs, online_ids))
                online_im = plot_tracking(
                    img, online_tlwhs, online_ids, frame_id=frame_id + 1, fps=1. / timer.average_time
                )
                vid_writer.write(online_im)

                frame_id += 1

            tracker.dump_cache()
            result_root = self.config.save_dir
            mkdirs(result_root)
            result_filename = osp.join(result_root, '{}.txt'.format(seq))
            write_results(result_filename, results)


    def _build(self):
        self._build_dir()
        self._build_encoder()
        self._build_model()
        if not self.config["eval_mode"]:
            self._build_train_loader()
            self._build_optimizer()

        print("> Everything built. Have fun :)")

    def _build_dir(self):
        self.model_dir = osp.join("./experiments",self.config.eval_expname)
        self.log_writer = SummaryWriter(log_dir=self.model_dir)
        os.makedirs(self.model_dir,exist_ok=True)
        log_name = '{}.log'.format(time.strftime('%Y-%m-%d-%H-%M'))
        log_name = f"{self.config.dataset}_{log_name}"

        log_dir = osp.join(self.model_dir, log_name)
        self.log = logging.getLogger()
        self.log.setLevel(logging.INFO)
        handler = logging.FileHandler(log_dir)
        handler.setLevel(logging.INFO)
        self.log.addHandler(handler)

        self.log.info("Config:")
        self.log.info(self.config)
        self.log.info("\n")
        self.log.info("Eval on:")
        self.log.info(self.config.dataset)
        self.log.info("\n")


        if self.config.eval_mode:
            epoch = self.config.eval_at
            checkpoint_dir = osp.join(self.model_dir, f"{self.config.dataset}_epoch{epoch}.pt")
            self.checkpoint = torch.load(checkpoint_dir, map_location = "cpu")

        print("> Directory built!")

    def _build_optimizer(self):
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.config.lr)
        self.scheduler = optim.lr_scheduler.ExponentialLR(self.optimizer,gamma=0.98)
        print("> Optimizer built!")

    def _build_encoder(self):
        self.encoder = History_motion_embedding()


    def _build_model(self):
        """ Define Model """
        config = self.config
        print(config)
        model = D2MP(config, encoder=self.encoder)

        self.model = model
        if not self.config.eval_mode:
            self.model = torch.nn.DataParallel(self.model, self.config.gpus).to('cuda')
        else:
            self.model = self.model.cuda()
            self.model = self.model.eval()

        if self.config.eval_mode:
            self.model.load_state_dict({k.replace('module.', ''): v for k, v in self.checkpoint['ddpm'].items()})

        print("> Model built!")

    def _build_train_loader(self):
        config = self.config
        data_path = config.data_dir
        self.train_dataset = DiffMOTDataset(data_path, config)

        self.train_data_loader = utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.config.batch_size,
            shuffle=True,
            num_workers=self.config.preprocess_workers,
            pin_memory=True
        )

    print("> Train Dataset built!")


> Train Dataset built!


In [20]:
config

{'eps': 0.001,
 'eval_mode': True,
 'lr': 0.0001,
 'data_dir': './outputs',
 'diffnet': 'HMINet',
 'interval': 5,
 'augment': True,
 'encoder_dim': 256,
 'tf_layer': 3,
 'epochs': 800,
 'batch_size': 2048,
 'seed': 123,
 'eval_every': 20,
 'gpus': [0, 1, 2, 3],
 'eval_at': 800,
 'det_dir': './outputs/detections/',
 'info_dir': './outputs',
 'reid_dir': 'weights/embedding',
 'save_dir': 'outputs/yolox_m_lt_ddm_1000eps_deeper_800_1rev',
 'eval_expname': 'mot_ddm_1000_deeper',
 'high_thres': 0.4,
 'low_thres': 0.1,
 'w_assoc_emb': 2.2,
 'aw_param': 1.7,
 'preprocess_workers': 16,
 'device': 'cuda',
 'eval_device': 'None',
 'video_path': 'WednesdayDance.mp4',
 'output_video_path': 'outputs/siren/img1',
 'output_txt_path': 'outputs',
 'weight': 'DanceTrack_yolox_s.tar',
 'yolox_type': 'yolox-s',
 'config': './configs/demo_test.yaml',
 'dataset': 'DanceTrack',
 'exp_name': 'demo_test'}

In [21]:
agent = DiffMOT(config)

2024-11-01 15:07:13 [INFO]: Config:
2024-11-01 15:07:13 [INFO]: {'eps': 0.001, 'eval_mode': True, 'lr': 0.0001, 'data_dir': './outputs', 'diffnet': 'HMINet', 'interval': 5, 'augment': True, 'encoder_dim': 256, 'tf_layer': 3, 'epochs': 800, 'batch_size': 2048, 'seed': 123, 'eval_every': 20, 'gpus': [0, 1, 2, 3], 'eval_at': 800, 'det_dir': './outputs/detections/', 'info_dir': './outputs', 'reid_dir': 'weights/embedding', 'save_dir': 'outputs/yolox_m_lt_ddm_1000eps_deeper_800_1rev', 'eval_expname': 'mot_ddm_1000_deeper', 'high_thres': 0.4, 'low_thres': 0.1, 'w_assoc_emb': 2.2, 'aw_param': 1.7, 'preprocess_workers': 16, 'device': 'cuda', 'eval_device': 'None', 'video_path': 'WednesdayDance.mp4', 'output_video_path': 'outputs/siren/img1', 'output_txt_path': 'outputs', 'weight': 'DanceTrack_yolox_s.tar', 'yolox_type': 'yolox-s', 'config': './configs/demo_test.yaml', 'dataset': 'DanceTrack', 'exp_name': 'demo_test'}
2024-11-01 15:07:13 [INFO]: 

2024-11-01 15:07:13 [INFO]: Eval on:
2024-11-01

C:\Users\ndp17\AppData\Local\Temp\ipykernel_1564\1441464184.py:197: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.checkpoint = torch.load(checkpoint_dir, map_location =

> Directory built!
{'eps': 0.001, 'eval_mode': True, 'lr': 0.0001, 'data_dir': './outputs', 'diffnet': 'HMINet', 'interval': 5, 'augment': True, 'encoder_dim': 256, 'tf_layer': 3, 'epochs': 800, 'batch_size': 2048, 'seed': 123, 'eval_every': 20, 'gpus': [0, 1, 2, 3], 'eval_at': 800, 'det_dir': './outputs/detections/', 'info_dir': './outputs', 'reid_dir': 'weights/embedding', 'save_dir': 'outputs/yolox_m_lt_ddm_1000eps_deeper_800_1rev', 'eval_expname': 'mot_ddm_1000_deeper', 'high_thres': 0.4, 'low_thres': 0.1, 'w_assoc_emb': 2.2, 'aw_param': 1.7, 'preprocess_workers': 16, 'device': 'cuda', 'eval_device': 'None', 'video_path': 'WednesdayDance.mp4', 'output_video_path': 'outputs/siren/img1', 'output_txt_path': 'outputs', 'weight': 'DanceTrack_yolox_s.tar', 'yolox_type': 'yolox-s', 'config': './configs/demo_test.yaml', 'dataset': 'DanceTrack', 'exp_name': 'demo_test'}


d:\anaconda\envs\DiffMOT\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


> Model built!
> Everything built. Have fun :)


In [34]:
if config["eval_mode"]:
    agent.eval()
else:
    agent.train()

2024-10-31 18:49:32 [INFO]: video save_path is ./outputs\siren\output.mp4
2024-10-31 18:49:32 [INFO]: Processing frame 0 (100000.00 fps)
C:\Users\ndp17\AppData\Local\Temp\ipykernel_1956\609851448.py:127: UserWarning: loadtxt: Empty input file: "./outputs/detections/siren/00001.jpg/00001.txt"
  dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]
C:\Users\ndp17\AppData\Local\Temp\ipykernel_1956\609851448.py:127: UserWarning: loadtxt: Empty input file: "./outputs/detections/siren/00002.jpg/00002.txt"
  dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]
C:\Users\ndp17\AppData\Local\Temp\ipykernel_1956\609851448.py:127: UserWarning: loadtxt: Empty input file: "./outputs/detections/siren/00003.jpg/00003.txt"
  dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]
C:\Users\ndp17\AppData\Local\Temp\ipykernel_1956\609851448.py:127: UserWarning: loadtxt: Empty input file: "./outputs/detections/siren/00004.jp

siren
Number of detection frames: 11488
Number of images: 5744


C:\Users\ndp17\AppData\Local\Temp\ipykernel_1956\609851448.py:127: UserWarning: loadtxt: Empty input file: "./outputs/detections/siren/00011.jpg/00011.txt"
  dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]
C:\Users\ndp17\AppData\Local\Temp\ipykernel_1956\609851448.py:127: UserWarning: loadtxt: Empty input file: "./outputs/detections/siren/00012.jpg/00012.txt"
  dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]
C:\Users\ndp17\AppData\Local\Temp\ipykernel_1956\609851448.py:127: UserWarning: loadtxt: Empty input file: "./outputs/detections/siren/00013.jpg/00013.txt"
  dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]
C:\Users\ndp17\AppData\Local\Temp\ipykernel_1956\609851448.py:127: UserWarning: loadtxt: Empty input file: "./outputs/detections/siren/00014.jpg/00014.txt"
  dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]
C:\Users\ndp17\AppData\Local\Temp\ipyker

Skipping invalid crop region: [  0  79   0 359]
Skipping invalid crop region: [  0  76   0 359]
Skipping invalid crop region: [  0  77   0 359]
Skipping invalid crop region: [  0  79   0 359]


2024-10-31 18:50:06 [INFO]: Processing frame 220 (6.76 fps)
2024-10-31 18:50:06 [INFO]: Processing frame 220 (6.76 fps)


...

2024-10-31 19:01:47 [INFO]: Processing frame 2840 (3.94 fps)
2024-10-31 19:01:47 [INFO]: Processing frame 2840 (3.94 fps)


Skipping invalid crop region: [  0   1   0 356]


2024-10-31 19:01:48 [INFO]: Processing frame 2850 (3.95 fps)
2024-10-31 19:01:48 [INFO]: Processing frame 2850 (3.95 fps)


Skipping invalid crop region: [  0  23   0 359]


C:\Users\ndp17\AppData\Local\Temp\ipykernel_1956\609851448.py:127: UserWarning: loadtxt: Empty input file: "./outputs/detections/siren/02858.jpg/02858.txt"
  dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]
C:\Users\ndp17\AppData\Local\Temp\ipykernel_1956\609851448.py:127: UserWarning: loadtxt: Empty input file: "./outputs/detections/siren/02859.jpg/02859.txt"
  dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]
C:\Users\ndp17\AppData\Local\Temp\ipykernel_1956\609851448.py:127: UserWarning: loadtxt: Empty input file: "./outputs/detections/siren/02860.jpg/02860.txt"
  dets = np.loadtxt(f_path, dtype=np.float32, delimiter=',').reshape(-1, 6)[:, 1:6]
2024-10-31 19:01:49 [INFO]: Processing frame 2860 (3.96 fps)
2024-10-31 19:01:49 [INFO]: Processing frame 2860 (3.96 fps)
2024-10-31 19:01:51 [INFO]: Processing frame 2870 (3.97 fps)
2024-10-31 19:01:51 [INFO]: Processing frame 2870 (3.97 fps)
2024-10-31 19:01:51 [INFO]: save res